In [14]:
import pandas as pd

# 1. Chargement des données
df = pd.read_csv("export_IA.csv")

# 2. Encodage de TransceiverClass
df["TransceiverClass"] = df["TransceiverClass"].map({'A': 0, 'B': 1})

# 3. Liste des colonnes candidates (hors ID, noms, géo, dates)
colonnes_candidates = [
    'SOG', 'COG', 'Heading', 'Length', 'Width', 'Draft',
    'Cargo', 'Status', 'TransceiverClass', 'VesselType'
]

# 4. Suppression des lignes avec valeurs manquantes
df_filtered = df[colonnes_candidates].dropna()

# 5. Calcul de la matrice de corrélation
correlation_matrix = df_filtered.corr(numeric_only=True)
correlation_with_target = correlation_matrix["VesselType"].drop("VesselType").sort_values(ascending=False)

# 6. Affichage des corrélations
print("Corrélation avec VesselType :\n")
print(correlation_with_target)

# 7. Seuil de corrélation minimale à garder (modifiable)
SEUIL = 0.5

# 8. Sélection automatique des colonnes pertinentes
variables_pertinentes = correlation_with_target[correlation_with_target.abs() >= SEUIL].index.tolist()

# 9. Sauvegarde de la liste dans un fichier txt (ou utilisable par import dans d'autres scripts)
with open("variables_selectionnees.txt", "w") as f:
    for var in variables_pertinentes:
        f.write(var + "\n")

print("\n✅ Variables retenues (corrélation >= ±" + str(SEUIL) + ") :")
print(variables_pertinentes)
print("\nListe enregistrée dans 'variables_selectionnees.txt'")

Corrélation avec VesselType :

Cargo               0.847490
Draft               0.585147
Width               0.545125
Length              0.510083
SOG                 0.008060
COG                -0.023292
Status             -0.098711
Heading            -0.481784
TransceiverClass         NaN
Name: VesselType, dtype: float64

✅ Variables retenues (corrélation >= ±0.5) :
['Cargo', 'Draft', 'Width', 'Length']

Liste enregistrée dans 'variables_selectionnees.txt'


In [15]:
# 1. Import des bibliothèques nécessaires
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# 2. Chargement du fichier CSV
df = pd.read_csv("Export_IA.csv")

# 3. Lecture des variables pertinentes sélectionnées par corrélation
with open("variables_selectionnees.txt", "r") as f:
    features = [line.strip() for line in f.readlines()]

# 4. Encodage de TransceiverClass si utilisée
if 'TransceiverClass' in features:
    df['TransceiverClass'] = df['TransceiverClass'].map({'A': 0, 'B': 1})

# 5. Cible à prédire
target = 'VesselType'

# 6. Filtrage du DataFrame avec les colonnes pertinentes + cible
features_and_target = features + [target]
df = df[features_and_target].dropna()

# 7. Affichage pour vérification
print("✅ Variables sélectionnées :", features)
print("\nAperçu des données filtrées :")
print(df.head())

# 8. Distribution de la cible
print("\nDistribution de VesselType :")
print(df['VesselType'].value_counts())

# 9. Séparation X / y
X = df[features]
y = df[target]

# 10. Visualisation si 'Length' est présente
if 'Length' in features:
    plt.figure(figsize=(8, 6))
    sns.boxplot(x='VesselType', y='Length', data=df)
    plt.title('Longueur des navires par type')
    plt.xticks(rotation=45)
    plt.savefig('boxplot_length_vesseltype.png')
    plt.close()
    print("📊 Boxplot sauvegardé dans boxplot_length_vesseltype.png")

# 11. Sauvegarde des fichiers préparés
X.to_csv('X_prepared.csv', index=False)
y.to_csv('y_prepared.csv', index=False)
print("\n✅ Données préparées sauvegardées dans X_prepared.csv et y_prepared.csv")


✅ Variables sélectionnées : ['Cargo', 'Draft', 'Width', 'Length']

Aperçu des données filtrées :
   Cargo  Draft  Width  Length  VesselType
0     83   10.0     28     183          80
1     60    2.3      9      47          60
2     60    1.9      9      30          60
3     70    5.0     20      92          70
4     80    6.5     25     160          80

Distribution de VesselType :
VesselType
70    85613
80    70061
60    31504
79     5544
71     5452
89     4491
82     2511
61     1840
74     1077
84      996
Name: count, dtype: int64
📊 Boxplot sauvegardé dans boxplot_length_vesseltype.png

✅ Données préparées sauvegardées dans X_prepared.csv et y_prepared.csv


In [16]:
# 1. Import des bibliothèques nécessaires
import pandas as pd
from sklearn.model_selection import train_test_split

# 2. Chargement des données préparées
X = pd.read_csv("X_prepared.csv")
y = pd.read_csv("y_prepared.csv")['VesselType']  # s'assurer qu'on extrait bien la colonne cible

# 3. Vérification des données
print("Aperçu des données X :")
print(X.head())
print("\nAperçu de la cible y :")
print(y.head())
print(f"\nDimensions de X : {X.shape}")
print(f"Dimensions de y : {y.shape}")

# 4. Découpage en ensembles d'entraînement et de test (80/20)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# 5. Affichage des dimensions
print("\nDécoupage terminé :")
print(f"X_train : {X_train.shape}")
print(f"X_test  : {X_test.shape}")
print(f"y_train : {y_train.shape}")
print(f"y_test  : {y_test.shape}")

# 6. Sauvegarde des ensembles
X_train.to_csv("X_train.csv", index=False)
X_test.to_csv("X_test.csv", index=False)
y_train.to_csv("y_train.csv", index=False)
y_test.to_csv("y_test.csv", index=False)

print("\n✅ Fichiers enregistrés : X_train.csv, X_test.csv, y_train.csv, y_test.csv")

Aperçu des données X :
   Cargo  Draft  Width  Length
0     83   10.0     28     183
1     60    2.3      9      47
2     60    1.9      9      30
3     70    5.0     20      92
4     80    6.5     25     160

Aperçu de la cible y :
0    80
1    60
2    60
3    70
4    80
Name: VesselType, dtype: int64

Dimensions de X : (209089, 4)
Dimensions de y : (209089,)

Découpage terminé :
X_train : (167271, 4)
X_test  : (41818, 4)
y_train : (167271,)
y_test  : (41818,)

✅ Fichiers enregistrés : X_train.csv, X_test.csv, y_train.csv, y_test.csv


In [17]:
# 1. Import des bibliothèques
import pandas as pd
import joblib
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# 2. Chargement des fichiers d'entraînement et de test
X_train = pd.read_csv("X_train.csv")
X_test = pd.read_csv("X_test.csv")
y_train = pd.read_csv("y_train.csv")["VesselType"]
y_test = pd.read_csv("y_test.csv")["VesselType"]

# 3. Normalisation (important pour certains modèles)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 4. Création et entraînement du modèle Random Forest
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train_scaled, y_train)

# 5. Prédictions
y_pred = model.predict(X_test_scaled)

# 6. Évaluation du modèle
print("📊 Rapport de classification :")
print(classification_report(y_test, y_pred))

# Matrice de confusion (affichée et sauvegardée)
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(10, 6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
plt.title("Matrice de confusion - Random Forest")
plt.xlabel("Prédit")
plt.ylabel("Réel")
plt.savefig("matrice_confusion.png")
plt.close()

# 7. Sauvegarde du modèle et du scaler pour la prédiction future
joblib.dump(model, "model.pkl")
joblib.dump(scaler, "scaler.pkl")

print("✅ Modèle sauvegardé dans model.pkl")
print("✅ Scaler sauvegardé dans scaler.pkl")
print("📁 Matrice de confusion enregistrée dans matrice_confusion.png")

📊 Rapport de classification :
              precision    recall  f1-score   support

          60       1.00      1.00      1.00      6301
          61       1.00      1.00      1.00       368
          70       1.00      1.00      1.00     17123
          71       1.00      1.00      1.00      1091
          74       1.00      1.00      1.00       215
          79       1.00      1.00      1.00      1109
          80       1.00      1.00      1.00     14012
          82       1.00      1.00      1.00       502
          84       1.00      1.00      1.00       199
          89       1.00      1.00      1.00       898

    accuracy                           1.00     41818
   macro avg       1.00      1.00      1.00     41818
weighted avg       1.00      1.00      1.00     41818

✅ Modèle sauvegardé dans model.pkl
✅ Scaler sauvegardé dans scaler.pkl
📁 Matrice de confusion enregistrée dans matrice_confusion.png


In [18]:
import pandas as pd
from sklearn.dummy import DummyClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
import joblib

# 1. Chargement
X_train = pd.read_csv("X_train.csv")
X_test = pd.read_csv("X_test.csv")
y_train = pd.read_csv("y_train.csv")["VesselType"]
y_test = pd.read_csv("y_test.csv")["VesselType"]

# 2. Normalisation
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 3. DummyClassifier (stratégie : "most_frequent")
model = DummyClassifier(strategy="most_frequent")
model.fit(X_train_scaled, y_train)

# 4. Prédiction et évaluation
y_pred = model.predict(X_test_scaled)

print("📊 Rapport de classification – DummyClassifier :")
print(classification_report(y_test, y_pred))

# 5. Matrice de confusion
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(10, 6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Reds")
plt.title("Matrice de confusion – DummyClassifier")
plt.xlabel("Prédit")
plt.ylabel("Réel")
plt.savefig("matrice_confusion_dummy.png")
plt.close()

# 6. Sauvegarde
joblib.dump(model, "model_dummy.pkl")
joblib.dump(scaler, "scaler_dummy.pkl")
print("✅ Dummy model sauvegardé dans model_dummy.pkl")


📊 Rapport de classification – DummyClassifier :
              precision    recall  f1-score   support

          60       0.00      0.00      0.00      6301
          61       0.00      0.00      0.00       368
          70       0.41      1.00      0.58     17123
          71       0.00      0.00      0.00      1091
          74       0.00      0.00      0.00       215
          79       0.00      0.00      0.00      1109
          80       0.00      0.00      0.00     14012
          82       0.00      0.00      0.00       502
          84       0.00      0.00      0.00       199
          89       0.00      0.00      0.00       898

    accuracy                           0.41     41818
   macro avg       0.04      0.10      0.06     41818
weighted avg       0.17      0.41      0.24     41818



c:\Users\adamb\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\adamb\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\adamb\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(ave

✅ Dummy model sauvegardé dans model_dummy.pkl


In [19]:
# MODELE_KNN.PY
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix
import joblib
import seaborn as sns
import matplotlib.pyplot as plt

# 1. Chargement
X_train = pd.read_csv("X_train.csv")
X_test = pd.read_csv("X_test.csv")
y_train = pd.read_csv("y_train.csv")["VesselType"]
y_test = pd.read_csv("y_test.csv")["VesselType"]

# 2. Normalisation
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 3. Modèle KNN
model = KNeighborsClassifier(n_neighbors=5)
model.fit(X_train_scaled, y_train)

# 4. Évaluation
y_pred = model.predict(X_test_scaled)
print("📊 Rapport de classification – KNN :")
print(classification_report(y_test, y_pred))

# 5. Matrice de confusion
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(10, 6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Greens")
plt.title("Matrice de confusion – KNN")
plt.xlabel("Prédit")
plt.ylabel("Réel")
plt.savefig("matrice_confusion_knn.png")
plt.close()

# 6. Sauvegarde
joblib.dump(model, "model_knn.pkl")
joblib.dump(scaler, "scaler_knn.pkl")

📊 Rapport de classification – KNN :
              precision    recall  f1-score   support

          60       1.00      1.00      1.00      6301
          61       1.00      1.00      1.00       368
          70       1.00      1.00      1.00     17123
          71       1.00      1.00      1.00      1091
          74       1.00      1.00      1.00       215
          79       1.00      1.00      1.00      1109
          80       1.00      1.00      1.00     14012
          82       1.00      1.00      1.00       502
          84       1.00      1.00      1.00       199
          89       1.00      1.00      1.00       898

    accuracy                           1.00     41818
   macro avg       1.00      1.00      1.00     41818
weighted avg       1.00      1.00      1.00     41818



['scaler_knn.pkl']

In [ ]:
import joblib
import numpy as np

def get_float_input(prompt):
    while True:
        try:
            return float(input(prompt))
        except ValueError:
            print("❌ Entrée invalide. Veuillez entrer un nombre.")

def get_int_input(prompt):
    while True:
        try:
            return int(input(prompt))
        except ValueError:
            print("❌ Entrée invalide. Veuillez entrer un entier.")

def predict_vessel_type():
    print("🔍 Chargement du modèle et du scaler...")
    model = joblib.load("model.pkl")
    scaler = joblib.load("scaler.pkl")

    print("\n Entrez les caractéristiques du navire :")

    # Seules les variables sélectionnées par corrélation sont demandées
    Cargo = get_int_input("Cargo : ")
    Draft = get_float_input("Draft : ")
    Width = get_float_input("Width : ")
    Length = get_float_input("Length : ")
    

    # Organiser l'entrée dans l'ordre des colonnes d'entraînement
    input_data = np.array([[Cargo, Draft, Width, Length,]])

    # Normalisation
    input_scaled = scaler.transform(input_data)

    # Prédiction
    prediction = model.predict(input_scaled)

    print(f"\n✅ Type de navire prédit (VesselType) : {prediction[0]}")

if __name__ == "__main__":
    predict_vessel_type()

🔍 Chargement du modèle et du scaler...

 Entrez les caractéristiques du navire :


In [ ]:
import joblib
import numpy as np

# Charger le modèle Dummy
model = joblib.load("model_dummy.pkl")
scaler = joblib.load("scaler_dummy.pkl")

# Saisie utilisateur
print("🔮 Prédiction avec Dummy Classifier")
print("Entrez les caractéristiques du navire :")
Cargo = int(input("Cargo : "))
Draft = float(input("Draft : "))
Width = float(input("Width : "))
Length = float(input("Length : "))


# Préparation
X_input = np.array([[Cargo, Draft, Width, Length]])
X_input_scaled = scaler.transform(X_input)

# Prédiction
prediction = model.predict(X_input_scaled)[0]
print(f"\n🛳️ (Dummy) Le navire est probablement de type : {prediction}")

In [ ]:
# PREDICTION_KNN.PY
import joblib
import numpy as np

# Chargement du modèle KNN et du scaler
model = joblib.load("model_knn.pkl")
scaler = joblib.load("scaler_knn.pkl")

# Saisie des caractéristiques du navire
print("Entrez les caractéristiques du navire :")
Cargo = int(input("Cargo : "))
Draft = float(input("Draft : "))
Width = float(input("Width : "))
Length = float(input("Length : "))


# Préparation des données
X_input = np.array([[Cargo, Draft, Width, Length]])
X_input_scaled = scaler.transform(X_input)

# Prédiction
prediction = model.predict(X_input_scaled)[0]

# Affichage
print(f"\n🛳️ Prédiction KNN : le navire est probablement de type : {prediction}")
